In [3]:

import os
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.widgets import MultiCursor, Slider
import matplotlib.figure as figs

import numpy as np
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d

import uncertainties as uc
from uncertainties import ufloat as uf
from uncertainties import umath as um
from uncertainties import unumpy as unp
%matplotlib tk

In [4]:
def timescanplot(fname, location, save, show, depletion=False):
    os.chdir(location)
    with open(fname, 'r') as f: file = f.readlines()
    
    skip = [num for num, line in enumerate(file) if 'ALL:' in line.split()]
    iterations = np.array([int(i.split()[-1]) for i in file if not len(i.strip())==0 and i.split()[0].startswith('#mass')])
    length = len(iterations)

    data = np.genfromtxt(fname, skip_header = skip[0]+1)

    cycle = int(len(data)/iterations.sum())
    time = data[:,1][:cycle]

    temp, temp1, temp2, mass, counts = [], [], [], [], []
    k = 0
    for i in range(len(iterations)):
        j = iterations[i]*cycle
        mass.append(data[:,0][k])
        counts.append(data[:,2][k:k+j])
        k += j
        
        for c in range(cycle):
            for l in range(iterations[i]):
                temp.append(counts[i][(l*cycle)+c])
            temp1.append(temp)
            temp = []
        temp2.append(temp1)
        temp1 = []
        
    mean = [[np.array(temp2[i][j]).mean() for j in range(cycle)]for i in range(length)]
    error = [[(np.array(temp2[i][j]).std()) for j in range(cycle)]for i in range(length)]
    mass, mean, error = np.array(mass), np.array(mean), np.array(error)

    if depletion: return mass, iterations, mean, error, time

    plt.figure(figsize=(15,5), dpi=100)

    for i in range(length):
        lg = "%i:%i"%(mass[i], iterations[i])
        plt.errorbar(time, mean[i],error[i],fmt='.-', label = lg)
        
    plt.title('Time Scan plot for %s'%fname)
    plt.xlabel('Time (ms)')
    plt.ylabel('Counts')
    plt.legend()
    plt.tight_layout()

    if save: plt.savefig(fname.split('.')[0]+'.png')
    if show: plt.show()
    
    plt.close()

In [184]:
import ctypes
user32 = ctypes.windll.user32
screensize = user32.GetSystemMetrics(0), user32.GetSystemMetrics(1)
w, h = screensize
dpi = 100

def depletionPlot(files, location, save, show, power_values, n):
    
    power_values = np.array(power_values)
    np.seterr(all='ignore')
    os.chdir(location)
    
    fig, axs = plt.subplots(figsize=(25, 10), dpi=70)
    plt.subplots_adjust(
        top = 0.95,
        bottom = 0.2,
        left = 0.05,
    )
    
    lg_fontsize = 15
    title_fontsize = 15
    lb_size = 15
    
    counts, stde = [], []
    for f in files:        
        mass, iterations, mean, error, time = timescanplot(f, location, save, show, depletion=True)
        time = time[1:]/1000
        mean = mean[0][1:]
        stde.append(error[0][1:])
        counts.append(mean)
        
    counts, stde = np.array(counts), np.array(stde)
    
    K_OFF, N = [], []
    K_OFF_err, N_err = [], []
    
    K_ON, Na0, Nn0 = [], [], []
    K_ON_err, Na0_err, Nn0_err = [], [], []
    
    for i in range(0, len(counts), 2):
        on = i
        off = i+1
        
        # making the error '0' value as very close to '0' 
        #since div by it makes it easier while fitting parameters
        stde[on][stde[on]==0]=10e-10
        stde[off][stde[off]==0]=10e-10
        
        #depletion values; y-axis
        depletion_on, depletion_on_err = counts[on], stde[on]
        depletion_off, depletion_off_err = counts[off], stde[off]
        
        #depletion values with uncertainities
        udepletion_on = unp.uarray(depletion_on, depletion_on_err)
        udepletion_off = unp.uarray(depletion_off, depletion_off_err)
        
        # power values; x-axis
        power_on = (power_values[on]*n*time)/1000. # divide by 1000 for mJ to J conversion 
        power_off = (power_values[off]*n*time)/1000.
        power_max = power_values.max()*n*time.max()/1000.
        x = np.linspace(0, power_max, num=len(time))
        
        axs.errorbar(power_off, depletion_off, yerr = depletion_off_err, fmt='o')
        axs.errorbar(power_on, depletion_on, yerr = depletion_on_err, fmt='o')
        
        ### finding parameters for fitting
        
        # depletion off
        def N_OFF(x, K_OFF, N):
            return (N)*np.exp(-K_OFF*x)
        
        K_OFF_init, N_init = 0, depletion_off.max()
        
        N_increase_bound_by = 0
        N_upper_bound = N_init + N_increase_bound_by
        
        pop_off, popc_off = curve_fit(
            N_OFF, power_off, depletion_off,
            sigma = stde[off],
            absolute_sigma = True,
            p0 = [K_OFF_init, N_init],
            bounds = [(-np.inf, 0), (np.inf, N_upper_bound)]
            )
        
        perr_off = np.sqrt(np.diag(popc_off))
        
        # off fitting variables         
        K_OFF.append(pop_off[0])
        N.append(pop_off[1])
        
        K_OFF_err.append(perr_off[0])
        N_err.append(perr_off[1])

        # depletion on
        def N_ON(X, Na0, Nn0, K_ON):
            x, K_OFF = X
            return Na0*np.exp(-K_ON*x)*np.exp(-K_OFF*x) + Nn0*np.exp(-K_OFF*x)

        #K_ON_init, Na0_init, Nn0_init = ()
        X = (power_on, pop_off[0])
        pop_on, popc_on = curve_fit(
            N_ON, X, depletion_on,
            sigma = stde[on],
            absolute_sigma = True,
            #p0 = [Na0_init, Nn0_init, K_ON_init]
            bounds = ([0,0,-np.inf], [pop_off[1], pop_off[1], np.inf])
        )
        perr_on = np.sqrt(np.diag(popc_on))
        
        #on fitting variables
        Na0.append(pop_on[0])
        Nn0.append(pop_on[1])
        K_ON.append(pop_on[2])
        
        Na0_err.append(perr_on[0])
        Nn0_err.append(perr_on[1])
        K_ON_err.append(perr_on[2])
    
    uK_OFF, uN = unp.uarray(K_OFF, K_OFF_err), unp.uarray(N, N_err)
    uK_ON, uNa0, uNn0 = unp.uarray(K_ON, K_ON_err), unp.uarray(Na0, Na0_err) , unp.uarray(Nn0, Nn0_err)
    
    ## depletion plot
    box0 = axs.get_position() ##[left, bottom, width, height]
    depletion_plot_position = [box0.x0+0.55, box0.y0-0.1, box0.width*0.45, box0.height*0.9]
    depletion_plot = plt.axes(depletion_plot_position)
    
    def Depletion(X, A):
        x, K_ON = X
        return A*(1-np.exp(-K_ON*x))
    
    uy_OFF = lambda x, uN, uK_OFF: uN*unp.exp(-uK_OFF*x)
    uy_ON = lambda x, uNa0, uNn0, uK_OFF, uK_ON : uNa0*unp.exp(-uK_ON*x)*unp.exp(-uK_OFF*x) + uNn0*unp.exp(-uK_OFF*x)
    
    A, A_err = [], []
    
    for i in range(len(N)):
        
        udepletion = 1 - uy_ON(x, uNa0[i], uNn0[i], uK_OFF[i], uK_ON[i])/uy_OFF(x, uN[i], uK_OFF[i])
        depletion, depletion_error = unp.nominal_values(udepletion), unp.std_devs(udepletion)
        
        #fitting for depletion
        X = (x, K_ON[i])
        pop_depletion, poc_depletion = curve_fit(
            Depletion, X, depletion, 
            sigma = depletion_error,
            absolute_sigma = True
        )
        
        A.append(pop_depletion[0])
        perr_A = np.sqrt(np.diag(poc_depletion))
        A_err.append(perr_A[0])

    uA = unp.uarray(A, A_err)

    def plot(i, l):
        # off plotting
        y_off0 = N_OFF(x, K_OFF[i], N[i])
        g_off0, = axs.plot(x, y_off0, label = 'K_OFF={:.2fP}/J, N={:.2fP}'.format(uK_OFF[i], uN[i]))

        # on plotting
        y_on0 = N_ON((x, K_OFF[i]), Na0[i], Nn0[i], K_ON[i])
        g_on0, = axs.plot(x, y_on0, label = 'K_ON={:.2fP}/J, N={:.2fP}, Na0={:.2fP}, Nn0={:.2fP}'.format(uK_ON[i], uNa0[i]+uNn0[i], uNa0[i], uNn0[i]))
        
        # deletion plot
        udepletion_new = 1 - uy_ON(x, uNa0[i], uNn0[i], uK_OFF[i], uK_ON[i])/uy_OFF(x, uN[i], uK_OFF[i])
        depletion_new, depletion_error_new = unp.nominal_values(udepletion_new), unp.std_devs(udepletion_new)

        depletion0, = depletion_plot.plot(x, depletion_new, '--')
        
        depletion_fitted = Depletion(X, A[i])
        depletion1, = depletion_plot.plot(x, depletion_fitted,
                                         label = 'A = {:.2fP}, K_ON = {:.2fP}/J'.format(uA[i], uK_ON[i])
                                         )
        
        # controlling fitting parameters
        axcolor = 'lightgoldenrodyellow'

        koff = plt.axes([l, 0.12, 0.2, 0.015], facecolor=axcolor) #[left, bottom, width, height]
        n = plt.axes([l, 0.10, 0.2, 0.015], facecolor=axcolor)

        kon = plt.axes([l, 0.08, 0.2, 0.015], facecolor=axcolor)
        na = plt.axes([l, 0.06, 0.2, 0.015], facecolor=axcolor)
        nn = plt.axes([l, 0.04, 0.2, 0.015], facecolor=axcolor)

        koff_slider = Slider(koff, '$K_{OFF}$', 0, K_OFF[i]+10, valinit = K_OFF[i])
        n_slider = Slider(n, 'N', 0, N[i]+(N[i]/2), valinit = N[i])

        kon_slider = Slider(kon, '$K_{ON}$', 0, K_ON[i]+10, valinit = K_ON[i])
        na_slider = Slider(na, '$Na_0$', 0, Na0[i]+(Na0[i]/2), valinit = Na0[i])
        nn_slider = Slider(nn, '$Nn_0$', 0, Nn0[i]+(Nn0[i]/2), valinit = Nn0[i])
                
        def update(val):

            koff = koff_slider.val
            ukoff = uf(koff, K_OFF_err[i])

            n = n_slider.val
            un = uf(n, N_err[i])

            kon = kon_slider.val
            ukon = uf(kon, K_ON_err[i])

            na = na_slider.val
            una = uf(na, Na0_err[i])

            nn = nn_slider.val
            unn = uf(nn, Nn0_err[i])

            yoff = N_OFF(x, koff, n)
            g_off0.set_ydata(yoff)

            yon = N_ON((x, koff), na, nn, kon)
            g_on0.set_ydata(yon)
             
            # depletion
            udepletion_new1 = 1 - uy_ON(x, una, unn, ukoff, ukon)/uy_OFF(x, un, ukoff)
            depletion_new1, depletion_error_new1 = unp.nominal_values(udepletion_new1), unp.std_devs(udepletion_new1)
            depletion0.set_ydata(depletion_new1)
              
            X = (x, kon)
            pop_depletion, poc_depletion = curve_fit(
                    Depletion, X , depletion_new1, 
                    sigma = depletion_error_new1, 
                    absolute_sigma = True
                )
            
            A_new1 = pop_depletion[0]
            perr = np.sqrt(np.diag(poc_depletion))[0]
            uA_new1 = uf(A_new1 , perr)
    
            depletion_fitted_new = Depletion(X, A_new1)
            depletion1.set_ydata(depletion_fitted_new)

            k = i*2
            legend.get_texts()[k].set_text('K_OFF={:.2fP}/J, N={:.2fP}'.format(ukoff, un))
            legend.get_texts()[k+1].set_text('K_ON={:.2fP}/J, N={:.2fP}, Na0={:.2fP}, Nn0={:.2fP}'.format(ukon, una+unn, una, unn))
            depletion_legend.get_texts()[i].set_text('A = {:.2fP}, K_ON = {:.2fP}/J'.format(uA_new1, ukon))

            fig.canvas.draw_idle()

        koff_slider.on_changed(update)
        n_slider.on_changed(update)

        kon_slider.on_changed(update)
        na_slider.on_changed(update)
        nn_slider.on_changed(update)
    
    widget_position = l = 0.05
    for i in range(len(N)):
        plot(i, l)
        l += 0.25

    ### setting labels
    title_depletion1 = '$N_{ON}(ntE)=N_{a0}e^{-k_{on}ntE}e^{-k_{off}ntE} + N_{n0}e^{-k_{off}ntE}$ ;\t$N_{OFF}(ntE)=(N)e^{-k_{off}ntE}$ ; $N = N_{a0}+ N_{n0}$'
    axs.set_title(title_depletion1, fontsize=title_fontsize)
    axs.set_xlabel('$n * t * E (Joule)$', fontsize= lb_size)
    axs.set_ylabel('Counts', fontsize= lb_size)

    axs.grid(True)
    box = axs.get_position()
    axs.set_position([box.x0, box.y0, box.width*0.6, box.height])
    legend = axs.legend(loc='center left', bbox_to_anchor=(1, 0.95), title=files, fontsize=lg_fontsize)
    legend.get_title().set_fontsize(lg_fontsize)
    
    depletion_plot.grid(True)
    depletion_legend = depletion_plot.legend(fontsize=lg_fontsize)
    depletion_plot.set_xlabel('$n * t * E (Joule)$', fontsize= lb_size)
    depletion_plot.set_ylabel('Relative abundance of active isomer', fontsize= lb_size)
    depletion_plot.set_title('$D(ntE) = 1-N_{ON}/N_{OFF}$ fitted with $D(ntE) = A(1-e^{K_{ON}*ntE})$', fontsize = title_fontsize)
    
    if save: plt.savefig("Depletion.pdf", bbox_inches='tight')
    if show: plt.show()
    #plt.close()
    
    #print(fig)
        
files1 = ['16_11_18-1.scan', '16_11_18-2.scan']
files2 = ['16_11_18-4.scan', '16_11_18-3.scan']
files3 = ['16_11_18-1.scan', '16_11_18-2.scan', '16_11_18-4.scan', '16_11_18-3.scan']
location = r'D:\FELion-Spectrum-Analyser\testing\_datas'

power1, power2, power3 = [12, 14.5], [22, 25], [12, 14.5, 22, 25]
depletionPlot(files3, location, save=True, show=True, power_values=power3,n=10)

In [140]:
import ctypes
user32 = ctypes.windll.user32
screensize = user32.GetSystemMetrics(0), user32.GetSystemMetrics(1)
w, h = screensize
r = w/h
x = 2.5/r
